1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля).


In [48]:

import requests
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs

client = MongoClient('mongodb://localhost:27017/')

In [49]:
#load job vacancy data from hh.ru
def load_hh(vacancy):
    vacancy = vacancy.replace(' ', '+')
    url = 'https://hh.ru/search/vacancy?text=' + vacancy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    response = requests.get(url, headers=headers)
    #soup = bs(response.text, 'lxml')
    soup = bs(response.text, 'html.parser')
    vacancy_items = soup.find_all('div', {'class': 'vacancy-serp-item-body'})
    vacancy_data = []

    for item in vacancy_items:
        vacancy_name = item.find('a', {'class': 'serp-item__title'}).getText()
        vacancy_link = item.find('a', {'class': 'serp-item__title'})['href']
        vacancy_salary = item.find('span', {'class': 'bloko-header-section-3'})
        if vacancy_salary is not None:
            vacancy_salary_currency = vacancy_salary.getText().split(" ")[-1]
            vacancy_salary = vacancy_salary.getText().replace('\xa0', '').replace('\u202f', '').replace(' ', '')
            if vacancy_salary.find('от') != -1:
                vacancy_salary_min = int(vacancy_salary.replace('от', '').replace(vacancy_salary_currency, ''))
                vacancy_salary_max = None
            elif vacancy_salary.find('до') != -1:
                vacancy_salary_min = None
                vacancy_salary_max = int(vacancy_salary.replace('до', '').replace(vacancy_salary_currency, ''))
            else:
                vacancy_salary_min = int(vacancy_salary.split('–')[0])
                vacancy_salary_max = int(vacancy_salary.split('–')[1].replace(vacancy_salary_currency, ''))
        else:
            vacancy_salary_min = None
            vacancy_salary_max = None
            vacancy_salary_currency = None
        vacancy_data.append({
            'name': vacancy_name,
            'link': vacancy_link,
            'salary_min': vacancy_salary_min,
            'salary_max': vacancy_salary_max,
            'currency': vacancy_salary_currency,
            'source': 'hh.ru'
        })
    return vacancy_data



In [71]:
def dbupdate(data):
    db = client.testdb
    for item in data:
        if len(list(db.vacancy.find({'name': item['name']}))) == 0:
            db.vacancy.insert_one(item)
            print(item['name']+" added to db")
        else:
            print(item['name']+" already in db")

In [73]:
vacancy = input("Введите название вакансии: ")
vacancy_data = load_hh(vacancy)
dbupdate(vacancy_data)

print(list(client.testdb.vacancy.find()))



Программист Python Junior added to db
Junior Data scientist (NLP) added to db
ML-специалист (Junior) added to db
Junior Python Developer added to db
Разработчик Python Junior added to db
ИТ-разработчик (уровень Junior) added to db
Junior Python Developer already in db
Junior Data Scientist (Стажер) added to db
Программист Python (Junior) added to db
Junior Python Backend Developer - разработчик Python added to db
Junior Data Engineer added to db
Программист С#, phyton (Junior) added to db
Computer Vision Engineer (junior/intern) added to db
Стажер-разработчик Python в Томске added to db
Junior machine learning engineer added to db
Data Scientist Junior/Junior+ added to db
Программист Python/Data Scientist (junior / middle) added to db
Junior/Middle Python Developer (Backend) - разработчик Питон added to db
Младший разработчик ПО added to db
Junior/Middle Backend-разработчик на Ruby added to db
[{'_id': ObjectId('640710daea9b50885f65f7b4'), 'name': 'Геофизик/петрофизик', 'link': 'https: